## Case Study 1
#### Note: This should be completed individually.

Assume you are new to the data science field, and you want to find out what real practitioners and soon-to-be data scientists are concerned about. One place where you may find such information is X (formerly known as Twitter). However, X users often use their real identities and may have reservations about sharing all their opinions publicly. Another place where such information maybe found is the datascience subreddit on Reddit.com (https://www.reddit.com/r/datascience/). Users are assumed to be anonymous and they are more likely to share their opinions without reservations. To find out common concerns among the datascience subreddit users, it might be a good idea to collect the top 100 posts in the subreddit in the year 2023. You might also collect the top 3 comments of each of those posts. In this case study, we will do exactly that. Specific details can be found in the next few cells. 

This data can be used for many different projects. However, we are only going to focus on the "data gathering" part. We will also do some cleaning.

**Note**: This case study contributes 10% to your overall grade.

## Step 1: 
###  15 points


**Description:** 

Learn about the **PRAW** package for Python and learn how you can use it to load reddit posts, comments etc. on a Jupyter Notebook. Do a Google search. You might find tutorials. It is okay to use them. You may need to use secret keys for this part. For that you will need to open a Reddit account. You can use a throwaway account for this purpose. Write your code in the cell below. Any code you write to retrieve data from Reddit can go there.

Here is a link to the PRAW documentation: https://praw.readthedocs.io/en/stable/#getting-started

**Grading criteria:** 

The code for this step must be correct. Otherwise, the next steps cannot be completed. In that case, the next steps will not be graded. If you receive a praw object from the data science subreddit, you will get full 15 points.' Other methods may be considered, but not encouraged.

In [1]:
# your code for step 1 goes here
! pip install praw

In [2]:
from datetime import *
import pandas as pd
import praw

reddit = praw.Reddit(
    client_id="nie3NCVXY1oPnnBXc9mJ3Q",
    client_secret="016KwctvgJzv0NeBVCCKicseRl-q5g",
    username="DataScientist2602",
    password="Santhosh26@",
    user_agent="DS501-CS1"
)


## Step 2: 
### 10 + 20 + 10 + 15 + 5 + 5 = 65 points

**Description**:
Once you have the mechanism in place to retrieve data from Reddit, you next step is to determine which parts of the data is necessary. For this case study, collect only the top posts from the year 2023. Also consider if the score of each post was above 50 or not. If the score was below 50, it might not have been an important post. Do not consider those posts. 

You may also observe that sometimes posts with memes or jokes get a lot of 'upvotes,' and because of that they may  have high scores, but they may not be useful for this case study. To address this problem, you will simply get rid of any post that has fewer than 5 words in the title. 

You will also notice that praw returns time as an integer. It is inconvenient for us to read time like that. You may want to convert the integer time to human readable time. You do not need to mention hours, minutes, or seconds. Just year, month and date is enough.

**Grading Criteria:**
* posts are only from the year 2023: 10 points
* the integer time format converted into year-month-day: 20 points
* only posts with scores more than 50 were considered: 10 points
* only post titles with more than 5 words were kept: 15 points
* minimum 100 posts were collected: 5 points
* three comments collected for each post: 5 points

Note: All six grading criteria can be satified by writing one line or many lines of code. It does not matter. As long as your code satisfies the six criteria (in one line or many lines), you will get full points. Otherwise, you will get partial credits.

Also note: In case the API does not allow you to collect 100 posts, you can collect 80 or 90 etc. In that case, please copy and paste the error message in a new cell below.

In [13]:
# your code for step 2 goes here
# create as many cells as necessary
subredditname="datascience" # Name of our SubReddit
'''To get posts from the beginning of the year,
   we import datetime package above and use datetime function to get the specified date.
'''
data=[]

def get_human_readable_time(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime("%Y-%m-%d")
# Get the current date
end_date = datetime(2023,12,31).timestamp()
start_date=datetime(2023,1,1).timestamp()

# Fetch submissions from the subreddit (fetch more than you need)
submissions = reddit.subreddit(subredditname).top(time_filter='year',limit=None)


In [16]:
# Fetch submissions from the subreddit
for submission in submissions:
    submission_time = submission.created_utc
    # Check if the submission is from the specified date range and score is more than 50 and title words are more than 5
    if start_date <= submission_time <= end_date and submission.score > 50 and len(submission.title.split()) >= 5:
        submission.comments.replace_more(limit=10)
        top_comments = []

        # skipping deleted comments and adding the next top comment
        for comment in submission.comments:
            if isinstance(comment, praw.models.Comment) and not comment.body == "[deleted]":
                top_comments.append(comment.body)
                
            # Break if we have collected 3 non-deleted top comments
            if len(top_comments) == 3:
                break

        # Ensure there are exactly 3 top comments
        top_comments.extend([""] * (3 - len(top_comments)))

        data.append([
            get_human_readable_time(submission_time),
            submission.score,
            submission.title,
            *top_comments
        ])

# Create a Pandas DataFrame from the collected data
df = pd.DataFrame(data, columns=["Date", "Post Score", "Post Title", "Top Comment 1", "Top Comment 2", "Top Comment 3"])

# Print the first few rows and the last few rows of the DataFrame
print(df.head())
print(df.tail())


         Date  Post Score                                         Post Title  \
0  2023-01-27        2636        As a hiring manager - this, this right here   
1  2023-04-26        1979  Pretty Accurate Chart to Clear Up Job Title Am...   
2  2023-01-20        1382  300,000+ Tech jobs have been vanished in the l...   
3  2023-02-27        1236    Which programming language is required for a...   
4  2023-07-17        1225                   XKCD Comic does machine learning   

                                       Top Comment 1  \
0  He also has a PhD in mathematics so I'm sure t...   
1  > _Yeah, well, that's just, like, your opinion...   
2  Based on your chart it's 200k+ in the past six...   
3  It's a shame you didn't copy the FAIRLY useful...   
4  Yes but how do you stir the pile correctly the...   

                                       Top Comment 2  \
0  I’m assuming he had a strong mathematical/stat...   
1  \nChart : Ambiguous\nJob Title Ambiguities : e...   
2  Someone els

In [18]:
df.count()

Date             561
Post Score       561
Post Title       561
Top Comment 1    561
Top Comment 2    561
Top Comment 3    561
dtype: int64

## Step 3: 
### 10 points

Save the data on your local disk. You may have used lists or similar data structures for the intial porcessing. Convert that data structure into a Pandas dataframe. Save the dataframe as a .csv file into your local disk. 

Here are the column details:

Column 1: Date

Column 2: Post score

Column 3: Post title

Column 4: Top comment 1

Column 5: Top comment 2

Column 6: Top comment 3

When you create the .csv file, it should have 101 rows (including column names) and 6 columns.

**Grading criteria:**
If your code produces a .csv file in the local disk in the same folder as the Jupyter Notebook file, you get full points. Otherwise, no point.

In [19]:
# your code for step 3 goes here
# create as many cells as necessary
df.to_csv("NityaPhaniSantosh_Oruganty_CaseStudy_1.csv", index=False,)

## Step 4:
### 10 points
#### Presentation slides:
   
Create presentation slides for this case study. The presentation slides should provide an overview of the problem you tried to solve, methods you have used (don't put actual code in the slides), and if you have discovered new insights from the data you have collected. You may put actual post titles or comments in the slide that you found insightful and interesting. The number of slides should be around 6-7 (no hard limit). Three of you will be randomly chosen and be asked to present your work in the class. You should be prepared to present your work for 5 mins.


**Notes on grading**: 5 points will be deducted if you are not prepared to present on the day of submission.

### What to submit:

All files should be named in the following format:

firstname_lastname_casestudy_1.pdf

firstname_lastname_casestudy_1.ipynb

firstname_lastname_casestudy_1.csv

etc.


Put the Jupyter Notebook file and the .csv file in a folder. Then convert your presentation slides to a PDF file and put it in the same folder. Zip the folder. After zipping, it should have the extension .zip. The name of the .zip file should be firstname_lastname_casestudy_1.zip . Upload the .zip file on Canvas.

